# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [113]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('./input/SINASC_RO_2019.csv')
print(sinasc.shape)

# Não há duplicados (removendo dados duplicados)
sinasc.drop_duplicates().shape

(27028, 69)


(27028, 69)

In [114]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [115]:
# Porcentagem de valores faltantes em cada coluna
sinasc.isna().sum() / len(sinasc.index)

ORIGEM        0.000000
CODESTAB      0.004255
CODMUNNASC    0.000000
LOCNASC       0.000000
IDADEMAE      0.000000
                ...   
munResUf      0.000000
munResLat     0.000037
munResLon     0.000037
munResAlt     0.000037
munResArea    0.000037
Length: 69, dtype: float64

In [116]:
# 3) seu código aqui
select_columns = [
    'LOCNASC', 
    'IDADEMAE', 
    'ESTCIVMAE', 
    'ESCMAE', 
    'QTDFILVIVO', 
    'GESTACAO', 
    'GRAVIDEZ', 
    'CONSULTAS', 
    'APGAR5'
]

sinasc_new = sinasc[select_columns]
sinasc_new

# count missing values
sinasc_new.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [117]:
# 4) seu código aqui
# novo dataset sem as linhas onde o valor da coluna APGAR5 é NaN
sinasc2 = sinasc_new[~sinasc_new.APGAR5.isna()]

sinasc2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [118]:
# 5) seu código aqui
sinasc2.head()
sinasc2['ESTCIVMAE'].fillna(9, inplace=True)
sinasc2['CONSULTAS'].fillna(9, inplace=True)

sinasc2.isna().sum()


/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/1271557862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['ESTCIVMAE'].fillna(9, inplace=True)
/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/1271557862.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['CONSULTAS'].fillna(9, inplace=True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [120]:
# 6) Seu código aqui
sinasc2['QTDFILVIVO'].fillna(0.0, inplace=True)
sinasc2['QTDFILVIVO'].notnull().sum()

/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/480345016.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['QTDFILVIVO'].fillna(0.0, inplace=True)


26925

In [121]:
sinasc2.shape

(26925, 9)

In [125]:
# 7) seu código aqui
sinasc2.head()

sinasc2.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [128]:
sinasc2[['ESCMAE', 'GESTACAO', 'GRAVIDEZ']]

,ESCMAE,GESTACAO,GRAVIDEZ
0,8 a 11 anos,37 a 41 semanas,Única
1,8 a 11 anos,37 a 41 semanas,Única
2,8 a 11 anos,37 a 41 semanas,Única
3,12 anos ou mais,37 a 41 semanas,Única
4,8 a 11 anos,37 a 41 semanas,Única
...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única
27024,8 a 11 anos,37 a 41 semanas,Única
27025,8 a 11 anos,37 a 41 semanas,Única
27026,8 a 11 anos,32 a 36 semanas,Única


In [142]:
# nessa coluna como existe o valor nenhuma fiz a subistituição por esse valor
sinasc2['ESCMAE'].fillna('Nenhuma', inplace=True)

sinasc2['ESCMAE'].unique()

/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/3810142144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['ESCMAE'].fillna('Nenhuma', inplace=True)


array(['8 a 11 anos', '12 anos ou mais', '1 a 3 anos', '4 a 7 anos',
       'Nenhuma'], dtype=object)

In [149]:
# nessa coluna subistitui pelo valor ignorado, pois não tem como identificar uma gestação
sinasc2['GESTACAO'].fillna('Ignorado', inplace=True)

sinasc2['GESTACAO'].isna().sum()

/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/1326523648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['GESTACAO'].fillna('Ignorado', inplace=True)


0

In [198]:
# como a porcentagem de dados nulos é menos de 1%
# e a probabilidade maior é que seja uma unica gravidez alterei 
# os valores nulos para 'Única'
sinasc2['GRAVIDEZ'].unique()

sinasc2['GRAVIDEZ'].value_counts()

percent_value = (sinasc2['GRAVIDEZ'].isna().sum() * 100) / sinasc2['GRAVIDEZ'].count()
percent_value

print(sinasc2['GRAVIDEZ'].unique())
print(sinasc2['GRAVIDEZ'].value_counts())
print("Percentual de valores nulos -> {:.2f} %".format(percent_value))

['Única' 'Dupla' nan 'Tríplice e mais']
Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64
Percentual de valores nulos -> 0.28 %


In [199]:
# alterado aqui
sinasc2['GRAVIDEZ'].fillna('Única', inplace=True)

/var/folders/wh/_2xtz5kd2knd10y0m8tmqhrc0000gp/T/ipykernel_16688/3990895366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc2['GRAVIDEZ'].fillna('Única', inplace=True)


In [201]:
# sem valores nulos
sinasc2.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [ ]:
# 8) seu código aqui


In [ ]:
# 9) seu código aqui
